# 1. DataFrame介绍：
DataFrame是Series是pandas中最重要的数据结构， DataFrame类似Excel数据表，从DataFrame中提取一行或一列时，会得到一个一维Series。
>info()方法返回 dataframe的基本信息，包括列信息（包括个数）、数据行数等。\
dtypes只返回列的信息。

In [1]:
import pandas as pd
df = pd.read_excel("data/stu_scores.xlsx")
df
df.info()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   stu_id     5 non-null      int64  
 1   stu_name   5 non-null      object 
 2   score      5 non-null      float64
 3   country    5 non-null      object 
 4   continent  5 non-null      object 
dtypes: float64(1), int64(1), object(3)
memory usage: 328.0+ bytes


stu_id         int64
stu_name      object
score        float64
country       object
continent     object
dtype: object

# 2.DataFrame的索引和列
DataFrame方法返回的是副本，原来的DataFrame没有任何变化。\
Dataframe的行标签被称为索引，如果不指定，pandas会自动创建从0开始的整数索引。\
reset_index将索引还原为普通的列，set_index可以设置一个新的索引。\
reindex会接管所有能够匹配新索引的行，而无法匹配的索引会引入含有空值（NaN）的行。\
使用sort_index可以按索引进行排序，使用sort_values按一列或多列进行排序。\
df.columns返回列的结合，可以设置列名。

In [3]:
df.index
df = df.reset_index().set_index("stu_id")
df.sort_values(["continent","country"])
df.columns
df
# df1 = df.reset_index().set_index("stu_id")
# df1.index

,index,stu_name,score,country,continent
stu_id,,,,,
1001,0,Jack,72.5,USA,America
1002,1,Alice,93.0,France,Europe
1003,2,Bob,84.5,China,Asia
1004,3,Lucy,89.0,Japan,Asia
1005,4,Tom,76.0,Germany,Europe


# 3.DataFrame选取数据
1.标签选取： 使用标签选取数据，使用loc方法来选取数据（意思是location），语法：loc[row_selection, column_selection] ，其中两个参数可以是数组，也可以是冒号表示的切片格式，在使用标签进行切片时，标签的区间包含区间首尾的两个标签（注意，这与数组的切片不一致，数组是前闭后开）。\
2.位置选取： 通过位置来选取数据使用iloc方法（意思是integer location）， 语法：iloc[row_selection, column_selection]，在使用切片时，iloc使用的是标准的半开半闭区间（左闭右开）。\
3.布尔选取：传给loc方法的也是可以布尔表达式，这里就像极了，SQL语句的WHERE条件，或者Excel里的筛选功能了。我在这里随便写一个例子。 例如亚洲的，分数大于85的人。 \
4.多级索引：MultiIndex是一种多级索引，按层次管理数据。


In [37]:
df.loc[1002:1004, "stu_name":"country"]
df.iloc[0:2,0:3]
# 需要注意用括号限制每个子条件，来改变优先级
df.loc[(df["continent"]=="Asia") & (df["score"] > 85) ,"stu_name": "continent"]
# 多级索引设置
df_multi = df.reset_index().set_index(["continent","country"])
df_multi.sort_index()

stu_id  index stu_name  score
continent country                               
America   USA        1001      0     Jack   72.5
Asia      China      1003      2      Bob   84.5
          Japan      1004      3     Lucy   89.0
Europe    France     1002      1    Alice   93.0
          Germany    1005      4      Tom   76.0

# 4.修改/替换数据
1.使用loc和iloc进行数据修改，先选取到一个数据集合，再使用=某值的方式，统一修改。 注意通过loc属性和iloc属性赋值时，原本的DataFrame是会被修改的。\
2.使用replace进行全局替换，或者单列替换 。 下边例子给出了全局和单列替换。\
3.同样使用loc和iloc也可以添加新列，添加方法与修改值一样（修改不存在的列）。

In [60]:
# 修改数据
df_update = df.copy()
df_update.loc[(df["score"] > 85),"stu_name"] = "XXX"
df_update

# 替换数据，continet列的字符串替换
df_update = df_update.replace("USA" , "U.S.")
df_update = df_update.replace({"continent" : {"Europe" : "Eur."}})
df_update

# 增加新列
df.loc[: , "age"] = [28, 32, 19, 42, 58]
df.loc[:, "birthday"] = 2022 - df["age"]
df

,index,stu_name,score,country,continent,age,birthday
stu_id,,,,,,,
1001,0,Jack,72.5,USA,America,28,1994
1002,1,Alice,93.0,France,Europe,32,1990
1003,2,Bob,84.5,China,Asia,19,2003
1004,3,Lucy,89.0,Japan,Asia,42,1980
1005,4,Tom,76.0,Germany,Europe,58,1964


# 5.缺失/重复值
1.缺失值，在pandas中表示缺失值一般使用None（字符串使用None）、np.nan（标量数值使用np.nan） 。关于缺失值，需要记录几个方法 dropna() 、dropna(how='all')、fillna()、isna() \
2.重复值： is_unique判断某列是否有重复值 ； drop_duplicates按列删除重复行，只保留一行。 unique()返回一个去重的列表。duplicated()打了个标记，标记每一行是否是重复的，第一次出现的不算重复，后边再重复出现就算重复。

In [61]:
# 缺失值操作
df_na_test = df.copy()
df_na_test.loc[1006 , : ] = None 
df_na_test.loc[1003,"score"] = None
df_na_test
# df_na_test.dropna(how = 'all')          #所有值都为None时，删掉该行
# df_na_test.dropna()                     #只要有None时，就删掉该行
df_na_test.fillna("Lost")

# 重复值确认
df["continent"].is_unique | df["score"].is_unique
df_duplicate = df.copy()
df_duplicate
df_duplicate.drop_duplicates(["continent","country"])        #按多列去重
df_duplicate["continent"].unique()
df_duplicate.duplicated()
df_duplicate["continent"].duplicated()

stu_id
1001    False
1002    False
1003    False
1004     True
1005     True
Name: continent, dtype: bool

# 6.算术运算
1.两个dataframe在进行运算时，只有共同的索引行、列会计算，其余都会变成None。如果希望保留其中一个的结果，可以使用add方法，并且使用fill_value参数替换None值。

In [65]:
rain_fall = pd.DataFrame(
    data=[[300.1,400.3,1000.5],[100.2,300.4,1100.6]],
    index = [0,1],
    columns = ["City1","City2","City3"]
)
more_rain_fall = pd.DataFrame(
    data=[[200,500],[100,700]],
    index = [1,2],
    columns = ["City1","City4"]
)
rain_fall + more_rain_fall             # 只有共同的索引+列名 才可以计算出结果，否则为None
rain_fall.add(more_rain_fall,fill_value=0)

,City1,City2,City3,City4
0,300.1,400.3,1000.5,NaN
1,300.2,300.4,1100.6,500.0
2,100.0,NaN,NaN,700.0


# 7.文本处理和应用函数
1.要在含有文本字符串的列上执行操作，需要使用str函数，并且可以访问python的字符串方法。\
2.DataFrame提供了applymap方法，该方法以自定义函数为参数，可以输入lambda函数做参数。\
3.视图和副本 ：（1）在原始dataframe中设置值，而不是在切片结果下设置 ；（2）如果想在切片后得到一个单独的dataframe，调用下copy方法。

In [15]:
users=pd.DataFrame(data=[" mARk", "jacK" ,"LuciFer ", "JOHN"]
                ,columns = ["name"])
users_clean = users.loc[: ,"name"].str.strip().str.capitalize()
# users_clean.str.startswith("J")
users.applymap(lambda x:x+"Tiktok")    #applymap函数需要应用到dataframe上

,name
0,mARkTiktok
1,jacKTiktok
2,LuciFer Tiktok
3,JOHNTiktok


# 8.多个dataframe合并、关联
1.用concat实现类似union all功能。axis = 0 实现了两个df的行级别合并，这里不会强行保证索引唯一。 concat可以实现多个df的合并，例如 : pd.concat([df1,df2,df3,...])。\
2.使用join实现两个DataFrame的关联。pandas在join函数中利用两个DataFrame将行对齐。outer代表full join 


In [16]:
# 使用concat实现dataframe合并
more_users_conti = pd.DataFrame(
    data = [[5,"Fibi",96, "Spain"] , [6,"Messi",72, "Argentina"]],
    columns = ["index","stu_name","score","country"],
    index = [1005,1006]
)
more_users_conti
pd.concat([df,more_users_conti],axis = 0)

# 使用Join 实现dataframe关联，通过索引关联
df_j1 = pd.DataFrame(
    data = [[1,2],[3,4],[5,6]],
    columns = ["A","B"]
)
df_j2 = pd.DataFrame(
    data = [[10,20],[30,40]] ,
    columns = ["C","D"],
    index = [1,3]
)
df_j1.join(df_j2,how="inner")
df_j1.join(df_j2,how="outer")

# 使用merge 指定关联列
df_j1.loc[:, "category"] = ["a","b","c"]
df_j2.loc[:,"category"] = ["d","b"]
df_j1.merge(df_j2,how="outer",on=["category"])




,A,B,category,C,D
0,1.0,2.0,a,NaN,NaN
1,3.0,4.0,b,30.0,40.0
2,5.0,6.0,c,NaN,NaN
3,NaN,NaN,d,10.0,20.0
